In [15]:
import numpy as np
import random
import json

class simpleSQL:
    def __init__(self) -> None:
        self.token=[]
        
    # def __init__(self,token = None) -> None:
    #     self.token=token
        
    def add(self,x):
        self.token.append(x)
        
    def toStr(self):
        for i in range(len(self.token)):
            print(str(self.token[i].value)+" ",end="")
        print()

class key:
    def __init__(self,value,type) -> None:
        self.value=value
        self.type=type
        # self.name=name


In [16]:
class foreign_constraint:
    def __init__(self,tb1,col1,tb2,col2) -> None:
        self.tb1=tb1
        self.col1=col1
        self.tb2=tb2
        self.col2=col2

In [17]:
class Table:
    def __init__(self,tb_name,col) -> None:
        self.name=tb_name
        self.col=col
        self.col_data_dis={}
    
    def addCharacteristics(self,col_name,data_dis):
        col_name_set=set(self.col[0:len(self.col)])
        if col_name not in col_name_set:
            print("error: add data characteristics failed. Col name not found.")
        else:
            self.col_data_dis[col_name]=data_dis

In [18]:
class DBschema:
    def __init__(self,tbs,cons=None) -> None:
        self.tables=tbs
        # self.tb_num=len(tbs)
        self.constraint=cons

In [19]:
# Random sampling in a given probability distribution graph
# ---- return the position(i.e. l+index) corresponding to the sample num.
def randNumGen(l,r,pdg):
    if r-l+1 != len(pdg):
        print("l is "+str(l)+" r is "+str(r)+" and pdg is "+str(pdg))
        print("error : pdg not match [l,r]")
        return l-1
    one_=sum(pdg)
    if abs(one_-1.0)>1e-5:
        print("error : probability sum "+str(one_)+" do not equal 1")
        return l-1
    pdg_=np.array(pdg)
    pdg_=[round(i*1000) for i in pdg_]
    maxv=sum(pdg_)
    num=random.randint(0,maxv-1)
    index=0
    for i in range(len(pdg_)):
        num=num-pdg_[i]
        if num<0:
            index=i
            break
    return l+index

##### test1 of randNumGen

In [20]:
dict={}
for i in range(1,6):
    dict[i]=0
for i in range(1,100000,1):
    # print(randNumGen(1,5,np.full((5),0.2)))
    # dict[randNumGen(1,5,np.full((5),0.2))]+=1
    dict[randNumGen(1,5,[0.1,0.2,0.3,0.4,0.0])]+=1
for i in range(1,6):
    print(dict[i])

9976
20119
29850
40054
0


##### test2 of randNumGen

In [21]:
dict={}
dict[0]=0
dict[1]=0
for i in range(1000):
    dict[randNumGen(0,1,np.array([0.7,0.3]))]+=1
print(dict[0])
print(dict[1])

657
343


In [22]:
# data access distribution
class feature:
    def __init__(self,rwRatio,avgtb) -> None:
        self.rwRatio=rwRatio
        self.avtb=avgtb
        

> ##### update table_name Set column_name = new_value Where column_name = some_value》
> ##### insert into table_name [(column1,column2,…)] value (value1,value2,…)
> ##### select {col_name} from table_name where col_name <=> constant

In [23]:
class SQLGen:
    def __init__(self,dbs) -> None:
        self.sqlSet=[]
        self.sql=simpleSQL()
        self.dbs=dbs

    def generate(self,condition):
        # feature1 : read/write ratio
        rw_choice=randNumGen(0,1,np.array([condition.rwRatio,1-condition.rwRatio]))
        # feature2 : average table num
        tb_num=randNumGen(1,2,[0.5,0.5])
        # feature3 : table data distribution
        tb_choice=self.dbs.tables[randNumGen(0,len(self.dbs.tables)-1,[1.0/len(self.dbs.tables) for i in range(len(self.dbs.tables))])]
        # feature4 : different query comparison constraint ratio
        cons=randNumGen(0,3,[1.0/4 for i in range(4)])
        # feature5 : query data domain distribution
        data=randNumGen(0,99,[1.0/100 for i in range(100)])
        if rw_choice==0:
            self.sql.add(key(value="select",type="keyword"))
            self.sql.add(key(value="*",type="colname"))
            self.sql.add(key(value="from",type="keyword"))
            self.sql.add(key(value=tb_choice.name,type="tbname"))
            self.sql.add(key(value="where",type="keyword"))
            self.sql.add(key(value=tb_choice.col[0].value,type="colname"))
            if cons==0:
                self.sql.add(key(value=">",type="compare"))
            else:
                if cons==1:
                    self.sql.add(key(value="<",type="compare"))
                else:
                    if cons==2:
                        self.sql.add(key(value="=",type="compare"))
                    else: 
                        if cons==3:
                            self.sql.add(key(value="!=",type="compare"))
                        else: 
                            pass
            self.sql.add(key(value=data,type="value"))
            self.sql.add(key(value=";",type="end"))
        else:
            self.sql.add(key(value="insert",type="keyword"))
            self.sql.add(key(value="into",type="keyword"))
            self.sql.add(key(value=tb_choice.name,type="tbname"))
            self.sql.add(key(value="value",type="keyword"))
            tmp="("
            for i in range(len(tb_choice.col)):
                if i!=0:
                    tmp+=","
                tmp+=str(randNumGen(0,99,[1.0/100 for i in range(100)]));
            tmp+=")"
            self.sql.add(key(value=tmp,type="value"))
            self.sql.add(key(value=";",type="end"))
        
        self.sqlSet.append(self.sql)
        return self.sql


In [24]:
jsonFile = open('input.json','r')
input = json.loads(jsonFile.read())
#print(input)
all_tables = []
cons = []
# load tabke name & column name
for table in input['Tables']:
    tb_name = table['Table Name']
    tb_cols = []
    for col in table['Table Columns']:
        col_name = col['Column Name']
        is_nullable = col['is_nullable']
        col_type = col['Column Type'][0]['data_type']
        tb_cols.append(key(col_name,col_type))
    all_tables.append(Table(tb_name,tb_cols))
# load constraints
for con in input['Constraints']:
    cons.append(input['Constraints'][con])

In [25]:
#tb=Table("lzz",[key("age",int),key("weight",int)])
dbs=DBschema(tbs=all_tables,cons=None)
sg=SQLGen(dbs=dbs)
fea=feature(cons[0],cons[1])
sg.generate(condition=fea)
sg.sql.toStr()

insert into tb3 value (82,27) ; 


In [26]:
def getNTable(dbs,n,pdg):
    all_tb=[dbs.tables[i] for i in range(len(dbs.tables))]
    # all_tb=[i for i in len(dbs.tables)]
    tb_set=set()
    if len(all_tb)<n:
        return 
    while len(tb_set)<n:
        x=randNumGen(0,len(all_tb)-1,pdg)
        if x!=-1:
            tb_set.add(dbs.tables[x])
        else:
            return -1,set()
    return 1,tb_set

##### test of getNTable

In [27]:
dict={}
for i in range(len(dbs.tables)):
    dict[dbs.tables[i].name]=0
for i in range(10000):
    bool,tb_set=getNTable(dbs=dbs,n=2,pdg=[0.1,0.3,0.6,0.0])
    for i in tb_set:
        # print(i.name)
        dict[i.name]+=1
for i in range(len(dbs.tables)):
    print(dict[dbs.tables[i].name])


2903
7864
9233
0


In [28]:
class SQLGen2(SQLGen):
    def generate(self,condition):
        # feature1 : read/write ratio
        rw_choice=randNumGen(0,1,np.array([condition.rwRatio,1-condition.rwRatio]))
        if rw_choice==0:
            # read
            # feature2 : average table num
            tb_num=randNumGen(1,2,[0.5,0.5])
            if tb_num==1:
                # feature3 : table data distribution
                tb_choice=self.dbs.tables[randNumGen(0,len(self.dbs.tables)-1,[1.0/len(self.dbs.tables) for i in range(len(self.dbs.tables))])]
                # feature4 : different query comparison constraint ratio
                cons=randNumGen(0,3,[1.0/4 for i in range(4)])
                # feature5 : query data domain distribution
                data=randNumGen(0,99,[1.0/100 for i in range(100)])
                self.sql.add(key(value="select",type="keyword"))
                self.sql.add(key(value="*",type="colname"))
                self.sql.add(key(value="from",type="keyword"))
                self.sql.add(key(value=tb_choice.name,type="tbname"))
                self.sql.add(key(value="where",type="keyword"))
                self.sql.add(key(value=tb_choice.col[0].value,type="colname"))
                if cons==0:
                    self.sql.add(key(value=">",type="compare"))
                elif cons==1:
                    self.sql.add(key(value="<",type="compare"))
                elif cons==2:
                    self.sql.add(key(value="=",type="compare"))
                elif cons==3:
                    self.sql.add(key(value="!=",type="compare"))
                else: 
                    pass
                self.sql.add(key(value=data,type="value"))
                self.sql.add(key(value=";",type="end"))
            elif tb_num==2:
                # JOIN
                bool,tbs=getNTable(dbs=dbs,n=2,pdg=np.full(2,0.5))
                if bool==-1:
                    print("fatal error : dbs table num not enough")
                else:
                    self.sql.add(key(value="select",type="keyword"))
                    self.sql.add(key(value="*",type="colname"))
                    self.sql.add(key(value="from",type="keyword"))
                    tbs_=list(tbs)
                    self.sql.add(key(value=tbs_[0].name,type="tbname"))
                    self.sql.add(key(value="join",type="keyword"))
                    self.sql.add(key(value=tbs_[1].name,type="tbname"))
                    self.sql.add(key(value="where",type="keyword"))
                    self.sql.add(key(value=tbs_[0].col[0].value,type="colname"))
                    self.sql.add(key(value="=",type="compare"))
                    self.sql.add(key(value=tbs_[1].col[0].value,type="colname"))
            else:
                pass
        elif rw_choice==1:
            tb_choice=self.dbs.tables[randNumGen(0,len(self.dbs.tables)-1,[1.0/len(self.dbs.tables) for i in range(len(self.dbs.tables))])]
            self.sql.add(key(value="insert",type="keyword"))
            self.sql.add(key(value="into",type="keyword"))
            self.sql.add(key(value=tb_choice.name,type="tbname"))
            self.sql.add(key(value="value",type="keyword"))
            tmp="("
            for i in range(len(tb_choice.col)):
                if i!=0:
                    tmp+=","
                tmp+=str(randNumGen(0,99,[1.0/100 for i in range(100)]));
            tmp+=")"
            self.sql.add(key(value=tmp,type="value"))
            self.sql.add(key(value=";",type="end"))
        else:
            pass
        self.sqlSet.append(self.sql)
        return self.sql
